In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,4,Rikitea,PF,79.02,clear sky,-23.1203,-134.9692,People ThankYou
1,8,Butaritari,KI,82.58,few clouds,3.0707,172.7902,Isles Sunset Lodge
2,9,Bathsheba,BB,77.61,few clouds,13.2167,-59.5167,Atlantis Hotel
3,13,Touros,BR,80.64,few clouds,-5.1989,-35.4608,INN NEW HORIZON
4,14,Atuona,PF,78.40,scattered clouds,-9.8000,-139.0333,Villa Enata


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt<dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}°F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
# 4a. Add a marker layer for each city to the map.
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                 max_intensity=300,point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
vacation_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,4,Rikitea,PF,79.02,clear sky,-23.1203,-134.9692,People ThankYou
1,8,Butaritari,KI,82.58,few clouds,3.0707,172.7902,Isles Sunset Lodge
2,9,Bathsheba,BB,77.61,few clouds,13.2167,-59.5167,Atlantis Hotel
3,13,Touros,BR,80.64,few clouds,-5.1989,-35.4608,INN NEW HORIZON
4,14,Atuona,PF,78.40,scattered clouds,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...,...
128,525,Piranhas,BR,75.83,overcast clouds,-16.4269,-51.8222,Planalto Hotel
129,526,Caravelas,BR,76.93,overcast clouds,-17.7125,-39.2481,Pousada dos Navegantes
130,528,Kavaratti,IN,77.70,overcast clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
131,534,Ca Mau,VN,77.90,broken clouds,9.1769,105.1500,Khách Sạn Mường Thanh Luxury Cà Mau


In [14]:
vacation_df.loc[vacation_df["City"]=="Kapaa",]


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,38,Kapaa,US,80.47,scattered clouds,22.0752,-159.319,Sheraton Kauai Coconut Beach Resort


In [15]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"]=="Kapaa"]
vacation_end = vacation_df.loc[vacation_df["City"]=="Kapaa"]
vacation_stop1 = vacation_df.loc[vacation_df["City"]=="Ewa Beach"]
vacation_stop2 = vacation_df.loc[vacation_df["City"]=="Kahului"]
vacation_stop3 = vacation_df.loc[vacation_df["City"]=="Hilo"]


In [16]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0] 
end = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]

In [21]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
city_itinerary = gmaps.directions_layer(
    start, end, waypoints=[stop1, stop2, stop3],
    travel_mode='DRIVING')
fig.add_layer(city_itinerary)
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,38,Kapaa,US,80.47,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
1,78,Ewa Beach,US,82.02,clear sky,21.3156,-158.0072,Uncle Junior Robert
2,454,Kahului,US,83.46,few clouds,20.8947,-156.4700,Maui Seaside Hotel
3,133,Hilo,US,78.62,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
4,38,Kapaa,US,80.47,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [19]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt<dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map.
locations = itinerary_df[["Lat","Lng"]]
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))